In [72]:
import undetected_chromedriver as uc
import time
import json
from tqdm import tqdm  # Progress bar
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd 
import os
from bs4 import BeautifulSoup
import utils
import hashlib
import re
from bs4 import BeautifulSoup
import requests
import random
from selenium import webdriver

In [104]:
driver = uc.Chrome(headless=False, use_subprocess=True)

In [103]:
driver.quit()

In [105]:
utils.login(driver)

In [6]:
# Step 2: read the profile details
with open("profiles.json", "r", encoding="utf-8") as file:
    profiles = json.load(file)

In [82]:
driver.get('https://www.instagram.com/p/DBNDq3YRW7S/')

In [ ]:
from bs4 import BeautifulSoup
import hashlib
import json

def load_comments():
    comments_section = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]')

    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)
        time.sleep(4)  

        try:
            load_more_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Load more comments')]")
            driver.execute_script("arguments[0].click();", load_more_button)  
            #print("Clicked 'Load more comments' button")
            time.sleep(5)  
        except NoSuchElementException:
            prev_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)
            time.sleep(5)
            new_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)

            if prev_height == new_height:  
                #print("Reached the end of comments.")
                break

def extract_comment(comment, hidden=False):
    try:
        # 1. Extract Username
        username_tag = comment.find("a", href=True)
        username = username_tag['href'].strip().replace("/", "") if username_tag else None

        # 2. Extract Profile Thumbnail (User Profile Picture)
        profile_img_tag = username_tag.find("img") if username_tag else None
        profile_thumbnail = profile_img_tag["src"] if profile_img_tag else None

        # 3. Extract Comment Text (Avoid timestamps, likes, and username)
        comment_text = None
        all_spans = comment.find_all("span", dir="auto")  # Look for spans with actual comment text
        for span in all_spans:
            span_text = span.get_text(strip=True)
            if span_text and username not in span_text and not span_text.endswith(("w", "d", "h", "m", "s")):
                comment_text = span_text  # Ensure it's not the timestamp
                break  # Get only the first valid comment text

        # 4. Extract Comment Image or GIF (Only images inside the comment, not profile picture)
        comment_image = None
        for img_tag in comment.find_all("img"):
            img_src = img_tag["src"]
            # Ensure the image is not the profile picture
            if profile_thumbnail and img_src != profile_thumbnail:
                comment_image = img_src
                break  # Only take the first relevant image in the comment

        # 5. Extract Likes
        num_likes = None
        for span in all_spans:
            text = span.get_text(separator=' ')
            if text.endswith('likes'):
                try:
                    num_likes = int(text.split(" ")[0])  # Extract first number
                except ValueError:
                    num_likes = None

        # 6. Extract Replies (Fixing this part)
        num_replies = None
        for span in all_spans:
            text = span.get_text(strip=True)
            if "replies" in text and "View all" in text:
                match = re.search(r"View all (\d+) replies", text)
                if match:
                    num_replies = int(match.group(1))
            elif text.startswith("View ") and text.endswith(" replies"):  # Handle cases like "View 2 replies"
                match = re.search(r"View (\d+) replies", text)
                if match:
                    num_replies = int(match.group(1))

        # 7. Extract Timestamp
        time_tag = comment.find("time")
        commented_time = time_tag.text.strip() if time_tag else None

        # 8. Generate Unique Comment ID
        comment_id_source = f"{username}-{commented_time}-{comment_text}"
        comment_id = hashlib.md5(comment_id_source.encode()).hexdigest() if comment_text else None

        return {
            "username": username,
            "profile_thumbnail": profile_thumbnail,
            "comment": comment_text,
            "comment_image": comment_image,
            "likes": num_likes,
            "replies": num_replies,
            "time": commented_time,
            "comment_id": comment_id,
            "hidden": hidden
        }
    except Exception as e:
        #print(f"Error extracting comment: {e}")
        return None

    
def get_comments(comment_objects, hidden=False):
    extracted_comments = [extract_comment(comment, hidden=hidden) for comment in comment_objects if extract_comment(comment, hidden=hidden)]
    return extracted_comments

def get_all_comments():
    # First, load or scroll to ensure comments are present
    load_comments()
    
    # Try to reveal hidden comments
    try:
        hidden_button = driver.find_element(By.XPATH, "//span[contains(text(), 'View hidden comments')]")
        hidden_button.click()
        time.sleep(2)  # or use explicit wait
    except NoSuchElementException:
        pass
    
    # Parse out everything
    load_comments()  # make sure everything is fully loaded or scrolled
    hidden_comments_data = []
    found_hidden_div = False
    
    try:
        # If there's a "Hidden by Instagram" indicator, stash it
        driver.find_element(By.XPATH, "//span[contains(text(), 'Hidden by Instagram')]")
        hidden_by_instagram_exists = True
    except NoSuchElementException:
        hidden_by_instagram_exists = False
    
    # Get the container for all comments
    reply_button = driver.find_element(By.XPATH, "//span[contains(text(), 'Reply')]")
    comments_container = reply_button.find_element(By.XPATH, './ancestor::*[9]')  # Adjust if needed    
    all_comments = comments_container.find_elements(By.XPATH, "./div")
    
    for comment in all_comments:
        comment_text = comment.text.strip()
        # Detect the "Hidden by Instagram" marker.  If it appears, flip the flag
        if "Hidden by Instagram" in comment_text:
            found_hidden_div = True
            continue
        
        # Actually parse the comment
        hidden_comments_data.append(
            extract_comment(
                BeautifulSoup(comment.get_attribute("outerHTML"), "html.parser"),
                hidden=found_hidden_div
            )
        )
    
    return hidden_comments_data


def save_comments(postId, comments):
    filename = f"comments/{postId}_comments.json"
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(comments, file, indent=4, ensure_ascii=False)
    print(f"Comments saved to {filename}")  

In [ ]:
def get_post_metadata(postId):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for i in soup.find_all("a"):
        if "liked_by" in i['href']:
            likes = i.getText().replace(" likes", "").replace(",", "")
            break
    postTime = soup.find("time")['datetime']
    with open('post_metadata.csv', 'a') as f:
        f.writelines(f"{postId},{likes},{postTime}\n")

In [9]:
driver.get("https://www.instagram.com/p/DBNDq3YRW7S/")

In [50]:
for profile in tqdm(profiles):
    for post in tqdm(profile["posts"]):
        post = '/'.join(post.split("/")[2:])
        if post.split("/")[-2] not in [i.replace("_comments.json", "") for i in os.listdir("comments")]:
            print("scraped")

100%|██████████| 33/33 [00:00<00:00, 2504.02it/s]


scraped


100%|██████████| 43/43 [00:00<00:00, 3779.84it/s]


scraped


 52%|█████▏    | 13/25 [00:00<00:00, 66.66it/s]

...

100%|██████████| 45/45 [00:00<00:00, 3337.76it/s]


scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped


scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped


100%|██████████| 25/25 [00:00<00:00, 58.49it/s]

scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped
scraped


In [95]:
scraped = [i.replace("_comments.json", "") for i in os.listdir("comments")]

In [96]:
posts = []

for profile in profiles:
    for post in profile["posts"]:
        if post.split("/")[-2] not in scraped:
            posts.append(post)

In [97]:
len(posts)

26

In [109]:
for id in posts[11:]:
    print(f"https://www.instagram.com{id}")
    print()

https://www.instagram.com/adityamadiraju/reel/DFLpKTPv1L-/

https://www.instagram.com/adityamadiraju/reel/DFLM-6KOs3u/

https://www.instagram.com/adityamadiraju/reel/DFK094auhht/

https://www.instagram.com/adityamadiraju/reel/DFIoZPduidP/

https://www.instagram.com/adityamadiraju/reel/DFIg09Lu91i/

https://www.instagram.com/adityamadiraju/reel/DFHR1YeONaG/

https://www.instagram.com/adityamadiraju/reel/DFGJGwgOaE-/

https://www.instagram.com/adityamadiraju/p/DFAlpCeuytZ/

https://www.instagram.com/adityamadiraju/reel/DE_hPpAOhBj/

https://www.instagram.com/adityamadiraju/reel/DE_QUEVuQJt/

https://www.instagram.com/adityamadiraju/reel/DE-PUjoOIbd/

https://www.instagram.com/adityamadiraju/reel/DE8mB8huVeA/

https://www.instagram.com/adityamadiraju/reel/DE5N8k8u1XN/

https://www.instagram.com/adityamadiraju/reel/DE48DyguUhf/

https://www.instagram.com/adityamadiraju/reel/DE3jOYOOOXY/



In [106]:
for post in tqdm(posts[11:]):
    post = '/'.join(post.split("/")[2:])
    driver.get(f"https://www.instagram.com/{post}")
    time.sleep(3)
    # try except block: a function to save the post like counts, posting date and append to csv file
    # and MAYBE download the post here
    try:
        comments = get_all_comments()
        save_comments(post.split("/")[-2], comments)
        time.sleep(2)
        get_post_metadata(post.split("/")[-2])
    except:
        continue
    time.sleep(10)

 13%|█▎        | 2/15 [00:06<00:43,  3.38s/it]


KeyboardInterrupt: 

In [94]:
driver.quit()

In [64]:
# Step 4: get comments 
for profile in tqdm(profiles[7:]):
    for post in tqdm(profile["posts"]):
        post = '/'.join(post.split("/")[2:])
        if post.split("/")[-2] not in [i.replace("_comments.json", "") for i in os.listdir("comments")]:
            driver.get(f"https://www.instagram.com/{post}")
            time.sleep(3)
            # try except block: a function to save the post like counts, posting date and append to csv file
            # and MAYBE download the post here
            try:
                comments = get_all_comments()
                save_comments(post.split("/")[-2], comments)
                time.sleep(2)
                get_post_metadata(post.split("/")[-2])
            except:
                continue

 28%|██▊       | 5/18 [00:08<00:23,  1.79s/it]


KeyboardInterrupt: 

In [46]:
profiles[7]

{'num_posts': '35',
 'num_followers': '27.6K',
 'num_following': '99',
 'bio': 'shakhed Threads shakhedc Dancer 💃Professional Dancer  📲Tiktok 900k+ ~ shakhed 💌 ~ shakhedcollab@gmail.com  ',
 'profile_picture': 'https://scontent-fra5-1.cdninstagram.com/v/t51.2885-19/480513890_9623714077688760_2363666855345936709_n.jpg?stp=dst-jpg_s150x150_tt6&_nc_ht=scontent-fra5-1.cdninstagram.com&_nc_cat=100&_nc_oc=Q6cZ2AF5VQdNGLg7SoIagAgPGmUS6Rxg0Chdfw8SpjqFQm14M3HOSvrWBp3hPrGc7t6LCac&_nc_ohc=peGBHnmFjbYQ7kNvgG-eZce&_nc_gid=9ae6e7576e134120b4e6cdffd08842bb&edm=AP4sbd4BAAAA&ccb=7-5&oh=00_AYGKP_g4dw5cSk8erYJ_L2ypEjrFlegHDd4zEUUQVkYbNg&oe=67D3D057&_nc_sid=7a9f4b',
 'posts': ['/shakhedc/p/DEArjKAM5y3/',
  '/shakhedc/p/DDxeWYJR4SJ/',
  '/shakhedc/reel/DDtKqiiIw-C/',
  '/shakhedc/p/DDSUTdTM3mQ/',
  '/shakhedc/p/DCdWFYkoycl/',
  '/shakhedc/reel/DB8ttVcsMgD/',
  '/shakhedc/reel/DBbih8jsK3I/',
  '/shakhedc/reel/DBMEdViMldA/',
  '/shakhedc/p/DA6TIYYonMx/',
  '/shakhedc/reel/DAnpJidMCBA/',
  '/shakhedc/p/C-79Kh

In [179]:
len(os.listdir("comments"))

47

In [122]:
comments = get_all_comments()

save_comments('DBNDq3YRW7S', comments)


Reached the end of comments.
Reached the end of comments.
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Comments saved to comments/DBNDq3YRW7S_comments.json


In [47]:
pd.read_json("comments/DBNDq3YRW7S_comments.json")

,username,profile_thumbnail,comment,comment_image,likes,replies,time,comment_id,hidden
0,thecaydenbrown,https://scontent-fra3-2.cdninstagram.com/v/t51...,👏👏👏,NaN,NaN,NaN,21 w,1fca1730744cefc26bcc5963e7db9d2f,False
1,idyllstyle,https://scontent-fra5-2.cdninstagram.com/v/t51...,🇺🇲🙏🏽😍💪🏽💯🥹,NaN,NaN,NaN,21 w,c4dcfd946efead4cb9d4a9191fd24bf5,False
2,cljnyc,https://scontent-fra5-1.cdninstagram.com/v/t51...,🗳️🗳️🗳️💙💙💙,NaN,NaN,NaN,21 w,e5ad60d8efc42516ee12fce8cc3aa4c7,False
3,theunofbabies2020,https://scontent-fra5-2.cdninstagram.com/v/t51...,💙💙💙,NaN,NaN,NaN,21 w,8ecd7a31c1beb92f347327e6862edf95,False
4,bellforus,https://scontent-fra3-1.cdninstagram.com/v/t51...,I hope I get to vote for you one day!!!!,NaN,2.0,NaN,21 w,9b3337b4dd88ae67ccd087ff7c298a03,False
5,katylied67,https://scontent-fra5-2.cdninstagram.com/v/t51...,THIS!!!!🙌❤️🙌🔥,NaN,NaN,NaN,21 w,5c71a654ecea74acc1a0ff1212f33ed4,False
6,drtiffanysanders,https://scontent-fra5-2.cdninstagram.com/v/t51...,Good for you for getting the word out to remin...,NaN,7.0,NaN,21 w,ade17719a6dd340c75c537a2ce17de92,False
7,hotrod_5.0,https://scontent-fra3-1.cdninstagram.com/v/t51...,👏🏾👏🏾❤️👏🏾👏🏾,NaN,NaN,NaN,20 w,0652c5f4be8c0c787a395cf913a1787e,False
8,perci.whitmore,https://scontent-fra5-1.cdninstagram.com/v/t51...,👏👏👏👏👏,NaN,NaN,NaN,21 w,bde47b817e143bff8b6d797e2d9c5034,False
9,elianamarie777,https://scontent-fra3-2.cdninstagram.com/v/t51...,live laugh love mari 💗,NaN,NaN,NaN,21 w,48371eb6107b9255f0560927a14b5d14,False


In [110]:
import os
import json
import hashlib
import re
from bs4 import BeautifulSoup

# Directory containing the HTML files
HTML_DIR = "pages"  # Change this to the actual directory where your files are stored
OUTPUT_DIR = "comments"   # Change this if needed

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

def extract_comment(comment, hidden=False):
    """Extracts relevant comment information from the parsed HTML."""
    try:
        # 1. Extract Username
        username_tag = comment.find("a", href=True)
        username = username_tag['href'].strip().replace("/", "") if username_tag else None

        # 2. Extract Profile Thumbnail
        profile_img_tag = username_tag.find("img") if username_tag else None
        profile_thumbnail = profile_img_tag["src"] if profile_img_tag else None

        # 3. Extract Comment Text
        comment_text = None
        all_spans = comment.find_all("span", dir="auto")  # Extract spans with actual text
        for span in all_spans:
            span_text = span.get_text(strip=True)
            if span_text and username not in span_text and not span_text.endswith(("w", "d", "h", "m", "s")):
                comment_text = span_text  # Ignore timestamps
                break  # Take only the first valid comment text

        # 4. Extract Comment Image (if any)
        comment_image = None
        for img_tag in comment.find_all("img"):
            img_src = img_tag["src"]
            if profile_thumbnail and img_src != profile_thumbnail:
                comment_image = img_src
                break  # Take only the first relevant image

        # 5. Extract Likes
        num_likes = None
        for span in all_spans:
            text = span.get_text(separator=' ')
            if text.endswith('likes'):
                try:
                    num_likes = int(text.split(" ")[0])  # Extract number before 'likes'
                except ValueError:
                    num_likes = None

        # 6. Extract Replies Count
        num_replies = None
        for span in all_spans:
            text = span.get_text(strip=True)
            if "replies" in text and "View all" in text:
                match = re.search(r"View all (\d+) replies", text)
                if match:
                    num_replies = int(match.group(1))
            elif text.startswith("View ") and text.endswith(" replies"):
                match = re.search(r"View (\d+) replies", text)
                if match:
                    num_replies = int(match.group(1))

        # 7. Extract Timestamp
        time_tag = comment.find("time")
        commented_time = time_tag.text.strip() if time_tag else None

        # 8. Generate Unique Comment ID
        comment_id_source = f"{username}-{commented_time}-{comment_text}"
        comment_id = hashlib.md5(comment_id_source.encode()).hexdigest() if comment_text else None

        return {
            "username": username,
            "profile_thumbnail": profile_thumbnail,
            "comment": comment_text,
            "comment_image": comment_image,
            "likes": num_likes,
            "replies": num_replies,
            "time": commented_time,
            "comment_id": comment_id,
            "hidden": hidden
        }
    except Exception as e:
        print(f"Error extracting comment: {e}")
        return None

def process_html_file(file_path, post_id):
    """Parses an HTML file and extracts comments."""
    with open(file_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")

    all_comments = []
    found_hidden_div = False

    # Find all comment divs
    comment_divs = soup.find_all("div", class_="")  # Adjust this if needed for correct class

    for comment in comment_divs:
        comment_text = comment.text.strip()

        # Detect "Hidden by Instagram" marker
        if "Hidden by Instagram" in comment_text:
            found_hidden_div = True
            continue  # Skip the marker itself

        # Extract comment details
        extracted_comment = extract_comment(comment, hidden=found_hidden_div)
        if extracted_comment:
            all_comments.append(extracted_comment)

    return all_comments

def save_comments(post_id, comments):
    """Saves extracted comments to a JSON file."""
    output_path = os.path.join(OUTPUT_DIR, f"{post_id}_comments.json")
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(comments, file, indent=4, ensure_ascii=False)
    print(f"Saved {len(comments)} comments to {output_path}")

def get_post_metadata(postId, html):
    soup = BeautifulSoup(html, 'html.parser')
    for i in soup.find_all("a"):
        if "liked_by" in i['href']:
            likes = i.getText().replace(" likes", "").replace(",", "")
            break
    postTime = soup.find("time")['datetime']
    with open('post_metadata.csv', 'a') as f:
        f.writelines(f"{postId},{likes},{postTime}\n")

def main():
    """Main function to process all HTML files."""
    for filename in os.listdir(HTML_DIR):
        if filename.endswith(".html"):
            post_id = os.path.splitext(filename)[0]  # Extract post ID from filename
            file_path = os.path.join(HTML_DIR, filename)
            print(f"Processing {filename}...")

            comments = process_html_file(file_path, post_id)
            save_comments(post_id, comments)
            get_post_metadata(post_id, open(file_path).read())

if __name__ == "__main__":
    main()


Processing DFAlpCeuytZ.html...
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting comment: 'in <string>' requires string as left operand, not NoneType
Error extracting commen